<a href="https://colab.research.google.com/github/guptaa98/Kaggle-Notebooks/blob/master/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#dataset is https://www.kaggle.com/kazanova/sentiment140 

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os


In [ ]:
project_data = pd.read_csv("/content/drive/My Drive/nlpassignment/assignment1.csv", encoding='latin-1')

In [ ]:
project_data.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
print("Number of data points in train data", project_data.shape)
print('-'*50)
print("The attributes of data :", project_data.columns.values)

Number of data points in train data (1599999, 6)
--------------------------------------------------
The attributes of data : ['target' 'ids' 'date' 'flag' 'user' 'text']


In [ ]:
#finding no of missing values of all features
def find_missing_values(df, columns):
    missing_val={}
    df_length=len(project_data)
    for column in columns:
        total_column_values=df[column].value_counts().sum()
        missing_val[column]=df_length-total_column_values
    return missing_val
missing_values=find_missing_values(project_data,project_data.columns)
missing_values

{'date': 0, 'flag': 0, 'ids': 0, 'target': 0, 'text': 0, 'user': 0}

In [ ]:
#project_data.drop(['date'],axis=1,inplace=True)

In [ ]:
approved_project = project_data['target'].values
project_data.drop(['target'], axis=1, inplace=True)
project_data.head(1)

,ids,date,flag,user,text
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [ ]:
# Data splitting

from sklearn.model_selection import train_test_split

# Splitting in train and test
X_train, X_test, y_train, y_test = train_test_split(project_data, approved_project, test_size=0.33, stratify=approved_project)

In [ ]:
#train
tokenization = ""
#wordnet_lemmatizer = WordNetLemmatizer()
for dataset in tqdm(X_train['text'].values):
        tokenization += dataset

100%|██████████| 1071999/1071999 [00:00<00:00, 1091740.00it/s]


In [ ]:
#test data
tokenization = ""
#wordnet_lemmatizer = WordNetLemmatizer()
for dataset in tqdm(X_test['text'].values):
        tokenization += dataset

100%|██████████| 528000/528000 [00:00<00:00, 1010644.76it/s]


In [ ]:
#TOP 10 most occuring words
from collections import Counter
split_it = tokenization.split()
Counter = Counter(split_it)
most_occur = Counter.most_common(10)
print(most_occur)

[('to', 182337), ('the', 160457), ('I', 158354), ('a', 120709), ('my', 91936), ('and', 90603), ('i', 80136), ('you', 70109), ('is', 69854), ('for', 69168)]


In [ ]:
#stemming on train
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()

preprocessed_text_train = []
# tqdm is for printing the status bar
for sentence in tqdm(X_train['text'].values):
    sent = word_tokenize(sentence)
    
    stem_word = ""
    for word in sent:
        stem_word =  stem_word + " " + stemmer.stem(word)
    
    preprocessed_text_train.append(stem_word)

[nltk_data] Downloading package punkt to /root/nltk_data...


  0%|          | 209/1071999 [00:00<08:33, 2085.91it/s]

[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 1071999/1071999 [08:42<00:00, 2052.15it/s]


In [ ]:
#stemming on test
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer= PorterStemmer()

preprocessed_text_test = []
# tqdm is for printing the status bar
for sentence in tqdm(X_test['text'].values):
    sent = word_tokenize(sentence)
    
    stem_word = ""
    for word in sent:
        stem_word =  stem_word + " " + stemmer.stem(word)
    
    preprocessed_text_test.append(stem_word)

  0%|          | 212/528000 [00:00<04:09, 2113.15it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 528000/528000 [04:20<00:00, 2028.37it/s]


In [ ]:
#lemmatization on train
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

lemmatized_text_train = []
for i in range (0,len(preprocessed_text_train)):
    
    input_str = word_tokenize(preprocessed_text_train[i])
    
    lemmatize_word = ""
    for word in input_str:
        lemmatize_word = lemmatize_word + " " + lemmatizer.lemmatize(word)
    
    lemmatized_text_train.append(lemmatize_word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#lemmatization on train
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

lemmatized_text_test = []
for i in range (0,len(preprocessed_text_test)):
    
    input_str = word_tokenize(preprocessed_text_test[i])
    
    lemmatize_word = ""
    for word in input_str:
        lemmatize_word = lemmatize_word + " " + lemmatizer.lemmatize(word)
    
    lemmatized_text_test.append(lemmatize_word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
final_preprocessed_text_train = []

for i in range (0,len(lemmatized_text_train)):
    sent = decontracted(lemmatized_text_train[i])
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent = re.sub("\S*\d\S*", "", sent).strip()
    sent = re.sub(r"@", '', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords)  
    final_preprocessed_text_train.append(sent.lower().strip())

In [ ]:
final_preprocessed_text_test = []

for i in range (0,len(lemmatized_text_test)):
    sent = decontracted(lemmatized_text_test[i])
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent = re.sub("\S*\d\S*", "", sent).strip()
    sent = re.sub(r"@", '', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords)  
    final_preprocessed_text_test.append(sent.lower().strip())

In [ ]:
for i in range (0,21):
    print(final_preprocessed_text_train[i])
    print("="*50)

joeymcintyr wonder go put quot one mani quot itun i hope onli cd i not
hope bro b move day it b much easier way let cross finger
oh yeah oreo time return
thank god friday
i got good pic everyon jonathanrknight favorit look like i go figur way go anoth show
timeladi believ wa news cnnfail took space import thing lt shake head gt
greenyclara hey clara yeah appar sold copi left actual i give comment
still havent got round watch supernatur
back home
kyleyleg soon enough i home
rxgellivictor oh no take care gelli
meliiann whi cri last night i sorri last nite
chelsr so excit rest intervi remind whi i not fan hi
dttucson not sure wa play shaq tweet know mall first person tag get autograph yada yada
sinequanon doe not realli make wan na watch not tempt
final found park i prob like left lunch
i miss boyfriend i wish i could right
http twitpic com misskittyfliff ca not pretend mips everyth
go earli night night twitter facebook youtube fmylife com bebo haha
it like eye vs ear thing
jlspinz boooo 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(final_preprocessed_text_train)
text_bow_train = vectorizer.transform(final_preprocessed_text_train)
text_bow_test = vectorizer.transform(final_preprocessed_text_test)
print("Shape of matrix after one hot encoding ",text_bow_train.shape)
print("Shape of matrix after one hot encoding ",text_bow_test.shape)

Shape of matrix after one hot encoding  (1071999, 25272)
Shape of matrix after one hot encoding  (528000, 25272)


In [ ]:
X_train['text']=text_bow_train
X_train.shape

(1071999, 5)

In [ ]:
X_test['text']=text_bow_test
X_test.shape

(528000, 5)

In [ ]:
X_train.head()

,ids,date,flag,user,text
170998,1963085974,Fri May 29 12:10:52 PDT 2009,NO_QUERY,debbier93,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
967061,1827747428,Sun May 17 11:21:42 PDT 2009,NO_QUERY,BekaHernandez,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1255065,1997215161,Mon Jun 01 16:27:29 PDT 2009,NO_QUERY,giiab,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1344907,2039972421,Thu Jun 04 23:17:09 PDT 2009,NO_QUERY,juwain,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
392238,2055101523,Sat Jun 06 08:42:02 PDT 2009,NO_QUERY,cdurham23,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."


In [ ]:
X_test.head()

,ids,date,flag,user,text
705263,2256307492,Sat Jun 20 12:34:38 PDT 2009,NO_QUERY,sweetmisary,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
788459,2325146666,Thu Jun 25 04:52:05 PDT 2009,NO_QUERY,christineporras,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
508014,2189056887,Mon Jun 15 23:08:41 PDT 2009,NO_QUERY,princessjasleen,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
767669,2300592010,Tue Jun 23 14:20:33 PDT 2009,NO_QUERY,AriSandcheeze,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
832468,1557601032,Sun Apr 19 05:03:37 PDT 2009,NO_QUERY,ipsy,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['flag'])
flag_bow_train = vectorizer.transform(X_train['flag'])
flag_bow_test = vectorizer.transform(X_test['flag'])
print("Shape of matrix after one hot encoding ",flag_bow_train.shape)
print("Shape of matrix after one hot encoding ",flag_bow_test.shape)

Shape of matrix after one hot encoding  (1071999, 1)
Shape of matrix after one hot encoding  (528000, 1)


In [ ]:
X_train['flag']=flag_bow_train
X_train.head()

,ids,date,flag,user,text
170998,1963085974,Fri May 29 12:10:52 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",debbier93,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
967061,1827747428,Sun May 17 11:21:42 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",BekaHernandez,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1255065,1997215161,Mon Jun 01 16:27:29 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",giiab,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1344907,2039972421,Thu Jun 04 23:17:09 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",juwain,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
392238,2055101523,Sat Jun 06 08:42:02 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",cdurham23,"(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."


In [ ]:
X_test['flag']=flag_bow_test
X_test.head()

,ids,date,flag,user,text
705263,2256307492,Sat Jun 20 12:34:38 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",sweetmisary,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
788459,2325146666,Thu Jun 25 04:52:05 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",christineporras,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
508014,2189056887,Mon Jun 15 23:08:41 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",princessjasleen,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
767669,2300592010,Tue Jun 23 14:20:33 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",AriSandcheeze,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
832468,1557601032,Sun Apr 19 05:03:37 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...",ipsy,"(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['user'])
user_bow_train = vectorizer.transform(X_train['user'])
user_bow_test = vectorizer.transform(X_test['user'])
print("Shape of matrix after one hot encoding ",user_bow_train.shape)
print("Shape of matrix after one hot encoding ",user_bow_test.shape)

Shape of matrix after one hot encoding  (1071999, 11757)
Shape of matrix after one hot encoding  (528000, 11757)


In [ ]:
X_train['user']=user_bow_train
X_train.head()

,ids,date,flag,user,text
170998,1963085974,Fri May 29 12:10:52 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
967061,1827747428,Sun May 17 11:21:42 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1255065,1997215161,Mon Jun 01 16:27:29 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1344907,2039972421,Thu Jun 04 23:17:09 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
392238,2055101523,Sat Jun 06 08:42:02 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."


In [ ]:
X_test['user']=user_bow_test
X_test.head()

,ids,date,flag,user,text
705263,2256307492,Sat Jun 20 12:34:38 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
788459,2325146666,Thu Jun 25 04:52:05 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
508014,2189056887,Mon Jun 15 23:08:41 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
767669,2300592010,Tue Jun 23 14:20:33 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
832468,1557601032,Sun Apr 19 05:03:37 PDT 2009,"(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=10)
vectorizer.fit(X_train['date'])
date_bow_train = vectorizer.transform(X_train['date'])
date_bow_test = vectorizer.transform(X_test['date'])
print("Shape of matrix after one hot encoding ",date_bow_train.shape)
print("Shape of matrix after one hot encoding ",date_bow_test.shape)

Shape of matrix after one hot encoding  (1071999, 72)
Shape of matrix after one hot encoding  (528000, 72)


In [ ]:
X_train['date']=date_bow_train
X_train.head()

,ids,date,flag,user,text
170998,1963085974,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
967061,1827747428,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1255065,1997215161,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1344907,2039972421,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
392238,2055101523,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."


In [ ]:
X_test['date']=date_bow_test
X_test.head()

,ids,date,flag,user,text
705263,2256307492,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
788459,2325146666,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
508014,2189056887,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
767669,2300592010,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
832468,1557601032,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."


In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X_train['ids'].values.reshape(-1,1))

X_train_ids = normalizer.transform(X_train['ids'].values.reshape(-1,1))
X_test_ids = normalizer.transform(X_test['ids'].values.reshape(-1,1))

In [ ]:
X_train['ids']=X_train_ids
X_test['ids']=X_test_ids

In [ ]:
X_train.head()

,ids,date,flag,user,text
170998,1.0,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
967061,1.0,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1255065,1.0,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
1344907,1.0,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."
392238,1.0,"(0, 10)\t1\n (0, 12)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(0, 2571)\t1\n (2, 3702)\t1\n (5, 6498)\t1...","(0, 3541)\t1\n (0, 8598)\t1\n (0, 9905)\t1..."


In [ ]:
X_test.head()

,ids,date,flag,user,text
705263,1.0,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
788459,1.0,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
508014,1.0,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
767669,1.0,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."
832468,1.0,"(0, 12)\t1\n (0, 20)\t1\n (0, 21)\t1\n (0...","(0, 0)\t1\n (1, 0)\t1\n (2, 0)\t1\n (3, 0...","(14, 7265)\t1\n (15, 1294)\t1\n (16, 6810)...","(0, 1566)\t1\n (0, 3318)\t1\n (0, 8598)\t1..."


In [ ]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

# Train data stack
X_tr = hstack((user_bow_train,flag_bow_train,text_bow_train,X_train_ids)).tocsr()


# Test Data Stack
X_te = hstack((user_bow_test,flag_bow_test,text_bow_test,X_test_ids)).tocsr()

print("Final Data matrix")
print(X_tr.shape, y_train.shape)

print(X_te.shape, y_test.shape)
print("="*100)

Final Data matrix
(1071999, 37031) (1071999,)
(528000, 37031) (528000,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_tr,y_train)
pred = clf.predict(X_te)

In [ ]:
#calculating accuracy
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)

0.7804147727272728

In [ ]:
#making confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[209277,  54723],
       [ 61218, 202782]])

In [ ]:
#precision, recall ad F1 Score 
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, pred, average='weighted')

(0.7805846026813408, 0.7804147727272728, 0.7803815405109188, None)